In [2]:
import numpy as np
#import pandas as pd
import pybamm
#import matplotlib.pyplot as plt

In [3]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [10]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

factor = 6.38
capacities = []
specific_capacities = []
l_p = 1e-4
thicknesses = np.linspace(1.0, 2.5, 11) * l_p
for l_n in thicknesses:
    e_ratio = np.around(l_n / l_p, 3)
    # Dimensions
    h = 0.137
    w = 0.207 / factor
    A = h * w
    l_s = 2.5e-5
    l1d = l_n + l_p + l_s
    vol = h * w * l1d
    vol_cm3 = vol * 1e6
    tot_cap = 0.0
    tot_time = 0.0
    fig, axes = plt.subplots(1, 2, sharey=True)
    I_mag = 1.01 / factor
    print("*" * 30)
    for enum, I_app in enumerate([-1.0, 1.0]):
        I_app *= I_mag
        # load model
        model = pybamm.lithium_ion.SPMe()
        # create geometry
        geometry = model.default_geometry
        # load parameter values and process model and geometry
        param = model.default_parameter_values
        param.update(
            {
                "Electrode height [m]": h,
                "Electrode width [m]": w,
                "Negative electrode thickness [m]": l_n,
                "Positive electrode thickness [m]": l_p,
                "Separator thickness [m]": l_s,
                "Lower voltage cut-off [V]": 2.8,
                "Upper voltage cut-off [V]": 4.7,
                "Maximum concentration in negative electrode [mol.m-3]": 25000,
                "Maximum concentration in positive electrode [mol.m-3]": 50000,
                "Initial concentration in negative electrode [mol.m-3]": 12500,
                "Initial concentration in positive electrode [mol.m-3]": 25000,
                "Current function [A]": I_app,
            }
        )
        param.process_model(model)
        param.process_geometry(geometry)
        s_var = pybamm.standard_spatial_vars
        var_pts = {
            s_var.x_n: 5,
            s_var.x_s: 5,
            s_var.x_p: 5,
            s_var.r_n: 5,
            s_var.r_p: 10,
        }
        # set mesh
        mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
        # discretise model
        disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
        disc.process_model(model)
        # solve model
        t_eval = np.linspace(0, 3600, 100)
        sol = model.default_solver.solve(model, t_eval)
        xpext = sol["X-averaged positive electrode extent of lithiation"]
        xnext = sol["X-averaged negative electrode extent of lithiation"]
        xpsurf = sol["X-averaged positive particle surface concentration"]
        xnsurf = sol["X-averaged negative particle surface concentration"]
        time = sol["Time [h]"]
        # Coulomb counting
        time_secs = sol["Time [s]"].entries
        time_hours = time(time_secs)
        dc_time = np.around(time_hours[-1], 3)
        # Capacity mAh
        cap = np.absolute(I_app * 1000 * dc_time)
        cap_time = np.absolute(I_app * 1000 * time_hours)
        axes[enum].plot(cap_time, xnext(time_secs), "r-", label="Average Neg")
        axes[enum].plot(cap_time, xpext(time_secs), "b-", label="Average Pos")
        axes[enum].plot(cap_time, xnsurf(time_secs), "r--", label="Surface Neg")
        axes[enum].plot(cap_time, xpsurf(time_secs), "b--", label="Surface Pos")
        axes[enum].set_xlabel("Capacity [mAh]")
        handles, labels = axes[enum].get_legend_handles_labels()
        axes[enum].legend(handles, labels)
        if I_app < 0.0:
            axes[enum].set_ylabel(
                "Extent of Lithiation, Elecrode Ratio: " + str(e_ratio)
            )
            axes[enum].title.set_text("Charge")
        else:
            axes[enum].title.set_text("Discharge")
        print(
            "Applied Current",
            I_app,
            "A",
            "Time",
            dc_time,
            "hrs",
            "Capacity",
            cap,
            "mAh",
        )
        tot_cap += cap
        tot_time += dc_time
    print("Negative : Positive electrode thickness", e_ratio)
    print("Total Charge/Discharge Time", tot_time, "hrs")
    print("Total Capacity", np.around(tot_cap, 3), "mAh")
    specific_cap = np.around(tot_cap, 3) / vol_cm3
    print("Total Capacity", specific_cap, "mAh.cm-3")
    capacities.append(tot_cap)
    specific_capacities.append(specific_cap)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(thicknesses / l_p, capacities)
ax2.plot(thicknesses / l_p, specific_capacities)
ax1.set_ylabel("Capacity [mAh]")
ax2.set_ylabel("Specific Capacity [mAh.cm-3]")
ax2.set_xlabel("Negative : Positive electrode thickness")
plt.show()

******************************
Applied Current -0.15830721003134796 A Time 0.513 hrs Capacity 81.2115987460815 mAh
Applied Current 0.15830721003134796 A Time 0.506 hrs Capacity 80.10344827586206 mAh
Negative : Positive electrode thickness 1.0
Total Charge/Discharge Time 1.0190000000000001 hrs
Total Capacity 161.315 mAh
Total Capacity 161.2954069059009 mAh.cm-3
******************************
Applied Current -0.15830721003134796 A Time 0.585 hrs Capacity 92.60971786833855 mAh
Applied Current 0.15830721003134796 A Time 0.589 hrs Capacity 93.24294670846393 mAh
Negative : Positive electrode thickness 1.15
Total Charge/Discharge Time 1.174 hrs
Total Capacity 185.853 mAh
Total Capacity 174.21602489509502 mAh.cm-3
******************************
Applied Current -0.15830721003134796 A Time 0.625 hrs Capacity 98.94200626959247 mAh
Applied Current 0.15830721003134796 A Time 0.671 hrs Capacity 106.22413793103448 mAh
Negative : Positive electrode thickness 1.3
Total Charge/Discharge Time 1.296 hrs
T

In [14]:
model.variables.search("initial")

No results for search using 'initial'. Best matches are ['Time', 'Electrolyte potential', 'Interfacial current density']


In [15]:
param

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Current function [A]': 0.15830721003134796,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.137,
 'Electrode width [m]': 0.03244514106583072,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x12c6e2160>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x12c6e2280>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 12500,
 'Initial concentration in positive electrode [mol.m-3]': 25000,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial temperature [K]': 2

In [20]:
chemistry = pybamm.parameter_sets.Chen2020

In [21]:
chemistry

{'chemistry': 'lithium-ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

In [22]:
default_parameter = model.default_parameter_values
default_parameter

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.13699999999999998,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x12c6e2160>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x12c6e2280>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial concentration in positive electrode [mol.m-3]': 30730.755438556498,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial tempera

In [23]:
parameter_values = pybamm.ParameterValues(chemistry=chemistry)
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.0053100000000000005,
 'Cell volume [m3]': 2.42e-05,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x157512790>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x157512af0>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initial concentration in positive electrode [mol.m-3]': 17038.0,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial temperature [K]': 298.15,
 'Inner SEI electron conductivity [S.m-1]': 8.95e-14,
 'Inn

In [24]:
parameter_values == default_parameter

False

In [25]:
parameter_values.search("init")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Initial concentration in electrolyte [mol.m-3]	1000.0
Initial concentration in negative electrode [mol.m-3]	29866.0
Initial concentration in positive electrode [mol.m-3]	17038.0
Initial inner SEI thickness [m]	2.5e-09
Initial outer SEI thickness [m]	2.5e-09
Initial temperature [K]	298.15


In [26]:
default_parameter.search("init")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Initial concentration in electrolyte [mol.m-3]	1000.0
Initial concentration in negative electrode [mol.m-3]	19986.609595075
Initial concentration in positive electrode [mol.m-3]	30730.755438556498
Initial inner SEI thickness [m]	2.5e-09
Initial outer SEI thickness [m]	2.5e-09
Initial temperature [K]	298.15


In [61]:
experiment = pybamm.Experiment(
[("Discharge at C/3 until 3.703 V",
 "Hold at 3.703 V until 5 mA",
 "Rest for 48 hours")], period="1 hours"
)

In [62]:
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
sim.solve()
sim.plot()

In [63]:
default_parameter.search("concentration")

Bulk solvent concentration [mol.m-3]	2636.0
EC initial concentration in electrolyte [mol.m-3]	4541.0
Initial concentration in electrolyte [mol.m-3]	1000.0
Initial concentration in negative electrode [mol.m-3]	19986.609595075
Initial concentration in positive electrode [mol.m-3]	30730.755438556498
Lithium interstitial reference concentration [mol.m-3]	15.0
Maximum concentration in negative electrode [mol.m-3]	24983.2619938437
Maximum concentration in positive electrode [mol.m-3]	51217.9257309275
Typical electrolyte concentration [mol.m-3]	1000.0


In [65]:
model.variables.search("concentration")

Average negative particle concentration
Average negative particle concentration [mol.m-3]
Average positive particle concentration
Average positive particle concentration [mol.m-3]
Electrolyte concentration
Electrolyte concentration [Molar]
Electrolyte concentration [mol.m-3]
Inner negative electrode SEI concentration [mol.m-3]
Inner positive electrode SEI concentration [mol.m-3]
Maximum negative particle concentration
Maximum negative particle concentration [mol.m-3]
Maximum negative particle surface concentration
Maximum negative particle surface concentration [mol.m-3]
Maximum positive particle concentration
Maximum positive particle concentration [mol.m-3]
Maximum positive particle surface concentration
Maximum positive particle surface concentration [mol.m-3]
Minimum negative particle concentration
Minimum negative particle concentration [mol.m-3]
Minimum negative particle surface concentration
Minimum negative particle surface concentration [mol.m-3]
Minimum positive particle conc

In [88]:
solution = sim.solution

In [67]:
c_s_n = solution["Average negative particle concentration [mol.m-3]"]

In [68]:
c_s_n.entries.shape

(54,)

In [69]:
plt.plot(c_s_n.entries)

In [70]:
c_s_n_ave = solution["X-averaged negative particle concentration [mol.m-3]"]

In [71]:
c_s_n_ave.entries.shape

(30, 54)

In [72]:
c_s_n_ave.entries[0]

array([29866.        , 20660.2571377 , 17284.39802093, 14167.86401236,
       13972.17170627, 13970.41120565, 13969.5455707 , 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067, 13969.54557067, 13969.54557067,
       13969.54557067, 13969.54557067])

In [73]:
c_s_n_ave_v = solution["Negative electrode volume-averaged concentration [mol.m-3]"]

In [75]:
plt.plot(c_s_n_ave_v.entries, label="a")
plt.plot(c_s_n.entries)
plt.legend()

In [83]:
model1 = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

In [84]:
experiment = pybamm.Experiment(
[("Rest for 48 hours")], period="1 hours"
)

In [85]:
sim1 = pybamm.Simulation(model1,experiment=experiment, parameter_values=parameter_values)
sim1.solve()
sim1.plot()

In [87]:
solution1 = sim1.solution

In [89]:
c_s_n_1 = solution1["Average negative particle concentration [mol.m-3]"]

In [90]:
c_s_n_ave_v_1 = solution1["Negative electrode volume-averaged concentration [mol.m-3]"]

In [143]:
c_s_p_1 = solution1["Average positive particle concentration [mol.m-3]"]

In [91]:
plt.plot(c_s_n_ave_v_1.entries, label="a")
plt.plot(c_s_n_1.entries)
plt.legend()

In [150]:
c_s_p = solution["Average positive particle concentration [mol.m-3]"]

In [151]:
c_s_n.entries[-1]

13969.545570668563

In [152]:
c_s_p.entries[-1]

37242.94565847709

In [156]:
model_SOC = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

In [157]:
parameter_values["Initial concentration in negative electrode [mol.m-3]"] = c_s_n.entries[-1].round(0)
parameter_values["Initial concentration in positive electrode [mol.m-3]"] = c_s_p.entries[-1].round(0)

In [158]:
experiment = pybamm.Experiment(
[("Rest for 10 seconds",
 "Charge at 1C for 10 seconds",
 "Rest for 10 seconds")], 
    period="0.1 seconds"
)

In [159]:
sim_SOC = pybamm.Simulation(model_SOC, experiment=experiment, parameter_values=parameter_values)
sim_SOC.solve()
sim_SOC.plot()

In [142]:
c_s_p.entries[-1].round(0)

13970.0

In [153]:
plt.plot(c_s_n.entries, label="negative", marker="o")
plt.plot(c_s_p.entries, labeL="positive", marker="o")
plt.plot(c_s_n_1.entries)
plt.plot(c_s_p_1.entries)

plt.legend()

# summary

In [5]:
#%%timeit

model = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

experiment = pybamm.Experiment(
[("Discharge at C/3 until 3.703 V",
 "Hold at 3.703 V until 5 mA",
 "Rest for 48 hours")], period="1 hours"
)

sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
sim.solve()
sim.plot()

solution = sim.solution
c_s_n = solution["Average negative particle concentration [mol.m-3]"]
c_s_p = solution["Average positive particle concentration [mol.m-3]"]
# print(c_s_n.entries[-1])
# print(c_s_p.entries[-1])

interactive(children=(FloatSlider(value=0.0, description='t', max=51.40879545104776, step=0.5140879545104776),…

In [10]:
model_SOC = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

parameter_values["Initial concentration in negative electrode [mol.m-3]"] = c_s_n.entries[-1].round(0)
parameter_values["Initial concentration in positive electrode [mol.m-3]"] = c_s_p.entries[-1].round(0)
parameter_values["Initial concentration in electrolyte [mol.m-3]"] = 1100.0

experiment = pybamm.Experiment(
[("Rest for 10 seconds",
 "Discharge at 10C for 10 seconds",
 "Rest for 10 seconds")], 
    period="0.1 seconds"
)

sim_SOC = pybamm.Simulation(model_SOC, experiment=experiment, parameter_values=parameter_values)
sim_SOC.solve()
sim_SOC.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=30.000000000000004, step=0.30000000000000004…

In [174]:
parameter_values.search("voltage")

Lower voltage cut-off [V]	2.5
Upper voltage cut-off [V]	4.4


In [8]:
parameter_values.search("initial")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Initial concentration in electrolyte [mol.m-3]	1000.0
Initial concentration in negative electrode [mol.m-3]	13970.0
Initial concentration in positive electrode [mol.m-3]	37243.0
Initial inner SEI thickness [m]	2.5e-09
Initial outer SEI thickness [m]	2.5e-09
Initial temperature [K]	298.15


In [16]:
parameter_values.search("potential")

Inner SEI open-circuit potential [V]	0.1
Outer SEI open-circuit potential [V]	0.8
SEI open-circuit potential [V]	0.4


# another way

In [22]:
model = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Chen2020
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

experiment = pybamm.Experiment(
[("Discharge at C/3 until 3.703 V",
 "Hold at 3.703 V until 5 mA",
 "Rest for 48 hours")], period="1 hours"
)

sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
sim.solve()
#sim.plot()

#solution = sim.solution
solution = sim.solve()

# MODEL RE-INITIALIZATION: #############################################################
# Now initialize the model with the solution of the charge, and then discharge with
# the US06 drive cycle
# We could also do this inplace by setting inplace to True, which modifies the original
# model in place
new_model = model.set_initial_conditions_from(solution, inplace=False)
########################################################################################

new_experiment = pybamm.Experiment(
[("Rest for 10 seconds",
 "Discharge at 9C for 10 seconds",
 "Rest for 10 seconds")], 
    period="0.1 seconds"
)

sim_SOC = pybamm.Simulation(new_model, experiment=new_experiment, parameter_values=parameter_values)
sim_SOC.solve()
sim_SOC.plot()


interactive(children=(FloatSlider(value=0.0, description='t', max=30.000000000000004, step=0.30000000000000004…

析出, 劣化も引き継いでいる -> 差が出ている？